##### Copyright 2018 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Automatically rewrite TF 1.x and compat.v1 API symbols

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/guide/migrate/upgrade">     <img src="https://www.tensorflow.org/images/tf_logo_32px.png">     TensorFlow.org で表示</a>
</td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/guide/migrate/upgrade.ipynb">     <img src="https://www.tensorflow.org/images/colab_logo_32px.png">     Google Colab で実行</a>
</td>
  <td><a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/guide/migrate/upgrade.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png"> GitHub でソースを表示</a></td>
  <td><a target="_blank" href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/guide/migrate/upgrade.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">ノートブックをダウンロード</a></td>
</table>


TensorFlow 2.x includes many API changes from TF 1.x and the `tf.compat.v1` APIs, such as reordering arguments, renaming symbols, and changing default values for parameters. Manually performing all of these modifications would be tedious and prone to error. To streamline the changes, and to make your transition to TF 2.x as seamless as possible, the TensorFlow team has created the `tf_upgrade_v2` utility to help transition legacy code to the new API.

注意: tf_upgrade_v2 は TensorFlow 1.13 以降（すべての TF 2.0 ビルドを含む）に自動的にインストールされています。

一般的な使用方法は以下のとおりです。

<pre class="devsite-terminal devsite-click-to-copy prettyprint lang-bsh">
tf_upgrade_v2 \
  --intree my_project/ \
  --outtree my_project_v2/ \
  --reportfile report.txt
</pre>

これにより、既存の TensorFlow 1.x Python スクリプトが TensorFlow 2.0 に変換され、アップグレード処理が高速化します。

The conversion script automates many mechanical API transformations, though many APIs cannot be automatically migrated. It is also not able to fully make your code compatible with TF2 behaviors and APIs. So, it is only a part of your migration journey.

## 互換性モジュール

Certain API symbols can not be upgraded simply by using a string replacement. Those that cannot be automatically upgraded will be mapped to their locations in the `compat.v1` module. This module replaces TF 1.x symbols like `tf.foo` with the equivalent `tf.compat.v1.foo` reference. If you are already using `compat.v1` APIs by importing TF via `import tensorflow.compat.v1 as tf`, the `tf_upgrade_v2` script will attempt to convert these usages to the non-compat APIs where possible. Note that while some `compat.v1` APIs are compatible with TF2.x behaviors, many are not. Therefore, it's recommended to manually proofread replacements and migrate them to new APIs in the `tf.*` namespace instead of `tf.compat.v1` namespace as quickly as possible.

TensorFlow 2.x で廃止されているモジュール（tf.flags や tf.contrib など）があるため、一部の変更は compat.v1 に切り替えても対応できません。このようなコードをアップグレードするには、追加のライブラリ（absl.flags など）を使用するか、tensorflow/addons にあるパッケージに切り替える必要があるかもしれません。


## 推奨アップグレード手順

The rest of this guide demonstrates how to use the symbol-rewriting script. While the script is easy to use, it is strongly recommended that you use the script as part of the following process:

1. 単体テスト: アップグレード対象のコードにカバレッジ率が適度な単体テストスイートを確実に用意します。このコードは Python で記述されているため、さまざまなミスから保護されることはありません。また、すべての依存物が TensorFlow 2.0 との互換性を確保できるようにアップグレード済みであることを確認してください。

2. **Install TensorFlow 1.15**: Upgrade your TensorFlow to the latest TensorFlow 1.x version, at least 1.15. This includes the final TensorFlow 2.0 API in `tf.compat.v2`.

3. 1.14 でのテスト: この時点で単体テストに合格することを確認します。単体テストはアップグレード中に何度も実行することになるため、安全な状態で開始することが重要です。

4. アップグレードスクリプトの実行: テストを含むソースツリー全体で tf_upgrade_v2 を実行します。これにより、TensorFlow 2.0 で利用できるシンボルのみを使用する形式にコードがアップグレードされます。廃止されたシンボルは tf.compat.v1 でアクセスできます。このようなシンボルは最終的には手動での対応が必要ですが、すぐに対応する必要はありません。

5. 変換後のテストを TensorFlow 1.14 で実行: コードは引き続き TensorFlow 1.14 で正常に動作するはずです。もう一度単体テストを実行してください。テストで何らかのエラーが発生する場合は、アップグレードスクリプトにバグがあります。その場合はお知らせください。

6. アップグレードレポートの警告とエラーを確認: このスクリプトは再確認が必要な変換や、必要な手動対応を説明するレポートファイルを書き出します。たとえば、残っているすべての contrib インスタンスを手動で削除する必要がある場合などです。RFC で詳細を確認してください。

7. **Install TensorFlow 2.x**: At this point it should be safe to switch to TensorFlow 2.x binaries, even if you are running with legacy behaviors

8. **Test with `v1.disable_v2_behavior`**: Re-running your tests with a `v1.disable_v2_behavior()` in the tests' main function should give the same results as running under 1.15.

9. **Enable V2 Behavior**: Now that your tests work using the TF2 binaries, you can now begin migrating your code to avoiding `tf.estimator`s and only using supported TF2 behaviors (with no TF2 behavior disabling). See the [Migration guides](https://tensorflow.org/guide/migrate) for details.

## Using the symbol-rewriting `tf_upgrade_v2` script


### Setup

Before getting started ensure that TensorFlow 2.x is installed.

In [ ]:
import tensorflow as tf

print(tf.__version__)

テスト対象のコードがある tensorflow/models git リポジトリをクローンします。

In [ ]:
!git clone --branch r1.13.0 --depth 1 https://github.com/tensorflow/models

### ヘルプを読む

スクリプトは TensorFlow と共にインストールされています。組み込みのヘルプは次のとおりです。

In [ ]:
!tf_upgrade_v2 -h

### TF1 のサンプルコード

単純な TensorFlow 1.0 のスクリプトは次のとおりです。

In [ ]:
!head -n 65 models/samples/cookbook/regression/custom_regression.py | tail -n 10

With TensorFlow 2.x installed it does not run:

In [ ]:
!(cd models/samples/cookbook/regression && python custom_regression.py)

### 単一ファイル

The script can be run on a single Python file:

In [ ]:
!tf_upgrade_v2 \
  --infile models/samples/cookbook/regression/custom_regression.py \
  --outfile /tmp/custom_regression_v2.py

コードの修正策が見つからない場合、スクリプトはエラーを出力します。 

### ディレクトリツリー

Typical projects, including this simple example, will use much more than one file. Typically want to update an entire package, so the script can also be run on a directory tree:

In [ ]:
# update the .py files and copy all the other files to the outtree
!tf_upgrade_v2 \
    --intree models/samples/cookbook/regression/ \
    --outtree regression_v2/ \
    --reportfile tree_report.txt

dataset.make_one_shot_iterator 関数に関する注意点が一つあります。

Now the script works in with TensorFlow 2.x:

Note that because the `tf.compat.v1` module is included in TF 1.15, the converted script will also run in TensorFlow 1.15.

In [ ]:
!(cd regression_v2 && python custom_regression.py 2>&1) | tail

## 詳細レポート

このスクリプトは、詳細な変更のリストも報告します。このサンプルでは、安全でない可能性のある変換が 1 つ検出され、ファイルの先頭で警告が表示されています。 

In [ ]:
!head -n 20 tree_report.txt

再度 Dataset.make_one_shot_iterator function に関して警告が 1 つ表示されていることに注意してください。

その他の場合、重要な変更の根拠が出力されます。

In [ ]:
%%writefile dropout.py
import tensorflow as tf

d = tf.nn.dropout(tf.range(10), 0.2)
z = tf.zeros_like(d, optimize=False)

In [ ]:
!tf_upgrade_v2 \
  --infile dropout.py \
  --outfile dropout_v2.py \
  --reportfile dropout_report.txt > /dev/null

In [ ]:
!cat dropout_report.txt

変更されたファイルの内容は次のとおりです。スクリプトがどのように引数名を追加し、移動および名前変更された引数を処理しているかに注目してください。

In [ ]:
!cat dropout_v2.py

大規模なプロジェクトでは、若干のエラーが発生する可能性があります。たとえば、deeplab モデルを変換します。

In [ ]:
!tf_upgrade_v2 \
    --intree models/research/deeplab \
    --outtree deeplab_v2 \
    --reportfile deeplab_report.txt > /dev/null

次のような出力ファイルが生成されました。

In [ ]:
!ls deeplab_v2

しかし、エラーが発生していました。レポートは、実行前に修正する必要があるものを正確に把握するのに役立ちます。最初の 3 つのエラーは次のとおりです。

In [ ]:
!cat deeplab_report.txt | grep -i models/research/deeplab | grep -i error | head -n 3

## "Safety" モード

この変換スクリプトには tensorflow.compat.v1 モジュールを使用するようにインポートを変更するだけの侵襲性の低い SAFETY モードもあります。

In [ ]:
!cat dropout.py

In [ ]:
!tf_upgrade_v2 --mode SAFETY --infile dropout.py --outfile dropout_v2_safe.py > /dev/null

In [ ]:
!cat dropout_v2_safe.py

As you can see this doesn't upgrade your code, but does allow TensorFlow 1 code to run against TensorFlow 2 binaries. Note that this does not mean your code is running supported TF 2.x behaviors!

## 注意事項

- Do not update parts of your code manually before running this script. In particular, functions that have had reordered arguments like `tf.math.argmax` or `tf.batch_to_space` cause the script to incorrectly add keyword arguments that mismap your existing code.

- The script assumes that `tensorflow` is imported using `import tensorflow as tf`, or `import tensorflow.compat.v1 as tf`.

- このスクリプトは引数の順序を変更しません。その代わり、キーワード引数を引数の順序が変更された関数に追加します。

- GitHub リポジトリ内の Jupyter ノートブックと Python ファイルをアップグレードするための便利なツールについては、[tf2up.ml](https://github.com/lc0/tf2up) をチェックしてください。

To report upgrade script bugs or make feature requests, please file an issue on [GitHub](https://github.com/tensorflow/tensorflow/issues).